# Epics Signal

In this notebook you will:

* Connect to some simulated hardware using `EpicsSignal`
* Explore the `EpicsSignal` interface.

Recommend Prerequisites:

* [Hello Python and Jupyter](./Hello%20Python%20and%20Jupyter.ipynb)

## Simulated Hardware
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

mini_beamline                    RUNNING   pid 4704, uptime 1:33:25
random_walk                      RUNNING   pid 4702, uptime 1:33:25
simple                           RUNNING   pid 4703, uptime 1:33:25
trigger_with_pc                  RUNNING   pid 4705, uptime 1:33:25


## Hello, EpicsSignal

An `EpicsSignal` is ophyd's representation of a single EPICS channel or a pair of channels, one readable and one writeable.

In [10]:
from ophyd import EpicsSignal

In [11]:
a = EpicsSignal('simple:A', name='a')

In [12]:
a

EpicsSignal(read_pv='simple:A', name='a', value=1, timestamp=1555269325.278924, auto_monitor=False, string=False, write_pv='simple:A', limits=False, put_complete=False)

In [13]:
a.name  # human-friendly label, which will be encoded in 'documents' emitted by bluesky

'a'

In [14]:
a.get()

1

In [18]:
a.put(3)
a.get()

3

In [16]:
a.read()

{'a': {'value': 3, 'timestamp': 1555280285.288266}}

In [17]:
a.describe()

{'a': {'source': 'PV:simple:A',
  'dtype': 'integer',
  'shape': [],
  'units': '',
  'lower_ctrl_limit': 0,
  'upper_ctrl_limit': 0}}

In [19]:
a.connected

True

In [20]:
a.wait_for_connection()

## Exercise

Instaniate an `EpicsSignal` that is connect to some made-up PV that does not exist, as in:

```py
broken = EpicsSignal('THIS_IS_NOT_A_THING', name='broken')
```

What does `broken.connected` do? What about `broken.wait_for_connection()`? And `broken.read()`?

## Read-only

The `EpicsSignalRO` represents a read-only signal. It can't be put to.

In [44]:
from ophyd import EpicsSignalRO

x = EpicsSignalRO('random_walk:x', name='x')

In [45]:
x.get()

0.9002995284235864

Try putting a value to `x` It will raise a `ReadOnlyError`.

## A read--write pair

Sometimes the readback and setpoint are different PVs. We can group them into one `EpicsSignal`.

In [ ]:
= EpicsSignal('trigger_with_pc:setpoint', write_pv='trigger_with_pc:readback')


## Accessing the PV name for debugging

When we have *many* signals in play, it can be useful to as a Signal which PV it is connected to (or attempting to connect to).

In [28]:
a.pvname  # PV name we gave above

'simple:A'

If ophyd is failing to connect, we can try to isolate the problem by using another Channel Access client like `caget` or `caproto-get`.

In [30]:
!caproto-get 'simple:A'

simple:A                                  [3]


We can add verbose output to learn more about which server this is from, etc.

In [31]:
!caproto-get -v 'simple:A'

[D 18:33:48.516 client:53] Registering with the Channel Access repeater.
[D 18:33:48.517 client:63] Searching for 'simple:A'....
[D 18:33:48.519 client:80] Search request sent to ('255.255.255.255', 5064).
[D 18:33:48.519 client:120] Found simple:A at ('192.168.86.21', 58091)
[I 18:33:48.568 client:168] simple:A connected
[D 18:33:48.568 client:184] Detected native data_type <ChannelType.LONG: 5>.
simple:A                                  [3]


In [32]:
!caproto-get -vvv 'simple:A'

[D 18:33:57.503 repeater:221] Another repeater is already running; will not spawn one.
[D 18:33:57.504 client:53] Registering with the Channel Access repeater.
[D 18:33:57.504 _broadcaster:74] Serializing 1 commands into one datagram
[D 18:33:57.504 _broadcaster:77] 1 of 1 RepeaterRegisterRequest(client_address='0.0.0.0')
[D 18:33:57.506 client:63] Searching for 'simple:A'....
[D 18:33:57.506 _broadcaster:74] Serializing 2 commands into one datagram
[D 18:33:57.506 _broadcaster:77] 1 of 2 VersionRequest(priority=0, version=13)
[D 18:33:57.506 _broadcaster:77] 2 of 2 SearchRequest(name='simple:A', cid=0, version=13, reply=5)
[D 18:33:57.508 client:80] Search request sent to ('255.255.255.255', 5064).
[D 18:33:57.508 _broadcaster:101] Received datagram from 192.168.86.21:5065 with 16 bytes.
[D 18:33:57.508 _broadcaster:109] 16 bytes -> RepeaterConfirmResponse(repeater_address='192.168.86.21')
[D 18:33:57.508 _broadcaster:101] Received datagram from 192.168.86.21:5064 with 40 bytes.
[D 18

## Exercise: Use `TAB` to explore `EpicsSignal` interface.

In [33]:
a.put_complete

False